### This notebook creates a supercell from a unit cell described by a cif-file

Note 1: The supercell atoms will not have unique names unless the connectivity function is run. The ones here are exclusively for carbamazepine (CBZ) and phenylbutazone (PBZ), but other custom naming functions can be added. 

Note 2: The connectivity function uses a distance matrix to search for neighboring atoms. This usually works for systems up to 8000 atoms on my PC (62.7 GiB), larger systems require larger memory.

Note 3: The default name for residues is CBZ as of now.

In [18]:
#import numpy
import numpy as np
from numpy.linalg import norm
import re

#import atomic simulation environment
import ase
from ase.io import read, write, cif
from ase.atoms import Atoms

#modified pdb writer
from read_pdb import write_proteindatabank

#custom functions
#from dihydrate_functions import get_nr_of_atoms, get_connectivity_list, create_arrays 
from cbz_functions import get_nr_of_atoms, get_connectivity_list, create_arrays

In [19]:
#read crystal structure from cif file

structure = read('form3.cif')

pos = structure.get_positions() #positions from cif file

sym = structure.get_chemical_symbols() #element symbols from cif file

#unit cell vectors
a = structure.cell[0]
b = structure.cell[1]
c = structure.cell[2]

chemform, atnr = get_nr_of_atoms(structure)
print(chemform, atnr) #check manually if this displays the correct number of atoms!
print(a, b, c)

C15H12N2O 30
[7.537 0.    0.   ] [ 0.    11.156  0.   ] [-0.69414884  0.         13.89467169]


/home/rahe7796/.local/lib/python3.8/site-packages/ase/io/cif.py:401: UserWarning: crystal system 'monoclinic' is not interpreted for space group Spacegroup(14, setting=1). This may result in wrong setting!
  warnings.warn(


In [22]:
# repeat unit cell

[na, nb, nc] = [6, 3, 3]
var = structure.repeat(rep=([na, nb, nc]))
varpos = var.get_positions()
symbol = var.get_chemical_symbols()
print(len(varpos))

#new unit cell vectors
array = var.get_cell_lengths_and_angles()

print(array)

6480
[45.222 33.468 41.736 90.    92.86  90.   ]


In [10]:
#get distance matrix

dist = var.get_all_distances(mic=True)
print(len(dist))

6480


In [11]:
print(len(var), len(varpos), len(symbol))

6480 6480 6480


In [13]:
#get atoms in correct order in residue, rearrange atom names and positions 
#if using dihydrate function get_connectivity returns three parameters, new_index, atom_names and h2o_counter
#and create_arrays needs six parameters passed to it, new_index, varpos, symbl, atnr, atom_names and h2o_counter

new_index, atom_names = get_connectivity_list(dist, var, varpos, symbol, atnr)
        
new_pos, symb, resnrs, atns = create_arrays(new_index, varpos, symbol, atnr, atom_names)

In [14]:
#create Atoms object and set new chemical symbols

atms = Atoms('O{}'.format(str(len(new_pos))),positions=new_pos)

atms.set_chemical_symbols(symb)

In [15]:
print(len(atms.get_positions())) #just to check the output

6480


In [16]:
#write supercell to PDB files

write_proteindatabank('supercell.pdb', atms, atns, resnrs, write_arrays=True)

In [17]:
#fix periodic boundary conditions in PDB file for supercell

x = '%.3f' % (na*norm(a)) #box x dimension
y = '%.3f' % (nb*norm(b)) #box y dimension
z = '%.3f' % (nc*norm(c)) #box z dimension

alpha = '%.2f' % ( np.rad2deg(np.arccos(np.dot(b,c)/(norm(b)*norm(c)))) ) #angle between b and c vector
beta = '%.2f' % ( np.rad2deg(np.arccos(np.dot(a,c)/(norm(a)*norm(c)))) ) #angle between a and c vector
gamma = '%.2f' % ( np.rad2deg(np.arccos(np.dot(a,b)/(norm(a)*norm(b)))) ) #angle between a and b vector

with open('supercell.pdb', 'r') as original: data = original.read()
with open('supercell.pdb', 'w') as modified: modified.write("CRYST1   {}   {}   {}  {}  {} {} P 1\n".format
                                                            (x,y,z,alpha,beta,gamma) + data)